In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pydicom
import os

sub_types = ['epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural']


base_dir = '/home/wkeeley/Desktop/Brain-Hemorrhaging'
image_dir = os.path.join(base_dir, 'Images')
train_dir = os.path.join(image_dir, 'Train_Images')
val_dir = os.path.join(image_dir, 'Validation_Images')

dcm_dir = os.path.join(base_dir, 'stage_1_train_images')
dcm_test_dir = os.path.join(base_dir, 'stage_1_test_images')

rsna_file = '/home/wkeeley/Desktop/Brain-Hemorrhaging/rsna-intracranial-hemorrhage-detection.zip'

sub_train_dirs = []
sub_val_dirs = []

for i in sub_types:
    sub_train_dirs.append(os.path.join(train_dir, i))
    sub_val_dirs.append(os.path.join(val_dir, i))



In [2]:
#Setup file hierarchy
for i in [base_dir, image_dir, train_dir, val_dir]:
    try:
        os.mkdir(i)
    except FileExistsError:
        pass

for i in sub_train_dirs:
    try:
        os.mkdir(i)
        os.mkdir(os.path.join(i, 'positive'))
        os.mkdir(os.path.join(i, 'negative'))
    except FileExistsError:
        pass

for i in sub_val_dirs:
    try:
        os.mkdir(os.path.join(i, 'positive'))
        os.mkdir(os.path.join(i, 'negative'))
    except FileExistsError:
        pass
    

In [3]:
data = pd.read_csv('stage_1_train.csv')

In [4]:
ids = []

for i in range(0, len(data), 6):
    img_id = data.iloc[i, 0][:12]
    ids.append(img_id)


In [5]:
ids

['ID_63eb1e259',
 'ID_2669954a7',
 'ID_52c9913b1',
 'ID_4e6ff6126',
 'ID_7858edd88',
 'ID_205506026',
 'ID_293f0cec8',
 'ID_12cb04d93',
 'ID_9aef16bb2',
 'ID_2802a8ae2',
 'ID_db5acfff8',
 'ID_ca62b5929',
 'ID_4419f8ae9',
 'ID_bcf1add28',
 'ID_aeda0804d',
 'ID_86efc4c17',
 'ID_877720a2f',
 'ID_ec7b5c7c9',
 'ID_8e2968e3f',
 'ID_fde0f077f',
 'ID_b2ba0706d',
 'ID_12fb979b2',
 'ID_4d4fd56d8',
 'ID_bf0c67657',
 'ID_066f36ad6',
 'ID_1024b5019',
 'ID_3fb8e224c',
 'ID_1dd5ecddd',
 'ID_d94d066a0',
 'ID_f8fd7b533',
 'ID_0706bacd4',
 'ID_2739292a5',
 'ID_f42d20382',
 'ID_02e0adea2',
 'ID_25321a1f8',
 'ID_94ee87654',
 'ID_5fc337950',
 'ID_0c0b804cb',
 'ID_85c8dead7',
 'ID_e2602081e',
 'ID_df33594dc',
 'ID_56e893e7c',
 'ID_975b56ce2',
 'ID_8eda1b9c2',
 'ID_4f4c47ae0',
 'ID_ea2a04ec1',
 'ID_646e3e896',
 'ID_29785bd15',
 'ID_f7356ca2b',
 'ID_e70495f58',
 'ID_34dd43f1a',
 'ID_e21609940',
 'ID_080c7a005',
 'ID_e7ac4ea57',
 'ID_ba18c312c',
 'ID_d42c02f69',
 'ID_d5667854d',
 'ID_87d2a77f1',
 'ID_ff1730610

In [14]:
from zipfile import ZipFile
num_images = 674262 #From data viz

rsna = ZipFile(rsna_file)

In [15]:
from windowing import window_image, get_windowing
import cv2


f = rsna.extract('stage_1_train_images/ID_94c3f5a91.dcm')
img_data = pydicom.read_file(f)
img = img_data.pixel_array
window_center, window_width, intercept, slope = get_windowing(img_data)
cv2.imwrite('test.jpg', window_image(img, 40, 80, intercept, slope))

True

In [20]:
from windowing import window_image, get_windowing
import PIL.Image as pil
from progressbar import *


widgets = ['Test: ', Percentage(), ' ', Bar(marker='0',left='[',right=']'),
           ' ', ETA(), ' ', FileTransferSpeed()]
pbar = ProgressBar(widgets=widgets)


for g in pbar(range(int(num_images*0.9))):
    i = ids[g]

    f = os.path.join('stage_1_train_images', i + '.dcm')
    file = rsna.extract(f)
    img_data = pydicom.read_file(file)
    window_center, window_width, intercept, slope = get_windowing(img_data)
    image = pydicom.read_file(file).pixel_array
    file_path = ''
    
    for j in sub_types:
        if j == sub_types[0]:
            img = window_image(image, 600, 2800, intercept, slope) #bone window
            
            
            if data.iloc[g, 1] == 1:
                file_path = os.path.join(os.path.join(sub_train_dirs[0], 'positive'), i + '.jpg')
            else:
                file_path = os.path.join(os.path.join(sub_train_dirs[0], 'negative'), i + '.jpg')
        elif j == sub_types[1]:
            img = window_image(image, 40, 80, intercept, slope) #brain window
            if data.iloc[g, 1] == 1:
                file_path = os.path.join(os.path.join(sub_train_dirs[1], 'positive'), i + '.jpg')
            else:
                file_path = os.path.join(os.path.join(sub_train_dirs[1], 'negative'), i + '.jpg') 
        elif j == sub_types[2]:
            img =  window_image(image, 40, 80, intercept, slope) #brain window
            if data.iloc[g, 1] == 1:
                file_path = os.path.join(os.path.join(sub_train_dirs[2], 'positive'), i + '.jpg')
            else:
                file_path = os.path.join(os.path.join(sub_train_dirs[2], 'negative'), i + '.jpg')
        elif j == sub_types[3]:
            img =  window_image(image, 40, 80, intercept, slope) #brain window
            if data.iloc[g, 1] == 1:
                file_path = os.path.join(os.path.join(sub_train_dirs[3], 'positive'), i + '.jpg')
            else:
                file_path = os.path.join(os.path.join(sub_train_dirs[3], 'negative'), i + '.jpg')
        elif j == sub_types[4]:
            img = window_image(image, 80, 200, intercept, slope) #subdural window
            if data.iloc[g, 1] == 1:
                file_path = os.path.join(os.path.join(sub_train_dirs[4], 'positive'), i + '.jpg')
            else:
                file_path = os.path.join(os.path.join(sub_train_dirs[4], 'negative'), i + '.jpg')
    
    p_img = pil.fromarray(img)
    p_img = p_img.resize((256, 256))
    img = np.asarray(p_img)
    cv2.imwrite(file_path, img)
    os.remove(file)

Test: 100% [0000000000000000000000000000000000000000] Time: 4:41:39  35.91  B/s


In [22]:
from windowing import window_image, get_windowing
import cv2

#Corrupted img -> ID_d4da787c8.jpg (img 645497)
for g in range(645500, num_images):
    i = ids[g]
    f = os.path.join('stage_1_train_images', i + '.dcm')
    file = rsna.extract(f)
    img_data = pydicom.read_file(file)
    window_center, window_width, intercept, slope = get_windowing(img_data)
    image = pydicom.read_file(file).pixel_array
    file_path = ''
    

    for j in sub_types:
        if j == sub_types[0]:
            img = window_image(image, 600, 2800, intercept, slope) #bone window
            
            if data.iloc[g, 1] == 1:
                file_path = os.path.join(os.path.join(sub_val_dirs[0], 'positive'), i + '.jpg')
                
            else:
                file_path = os.path.join(os.path.join(sub_val_dirs[0], 'negative'), i + '.jpg')
                #accidentally ran partially with train dirs instead of val dirs

        elif j == sub_types[1]:
            img = window_image(image, 40, 80, intercept, slope) #brain window
            if data.iloc[g, 1] == 1:
                file_path = os.path.join(os.path.join(sub_val_dirs[1], 'positive'), i + '.jpg')
            else:
                file_path = os.path.join(os.path.join(sub_val_dirs[1], 'negative'), i + '.jpg')
        elif j == sub_types[2]:
            img =  window_image(image, 40, 80, intercept, slope) #brain window
            if data.iloc[g, 1] == 1:
                file_path = os.path.join(os.path.join(sub_val_dirs[2], 'positive'), i + '.jpg')
            else:
                file_path = os.path.join(os.path.join(sub_val_dirs[2], 'negative'), i + '.jpg')
        elif j == sub_types[3]:
            img =  window_image(image, 40, 80, intercept, slope) #brain window
            if data.iloc[g, 1] == 1:
                file_path = os.path.join(os.path.join(sub_val_dirs[3], 'positive'), i + '.jpg')
            else:
                file_path = os.path.join(os.path.join(sub_val_dirs[3], 'negative'), i + '.jpg')
        elif j == sub_types[4]:
            img = window_image(image, 80, 200, intercept, slope) #subdural window
            if data.iloc[g, 1] == 1:
                file_path = os.path.join(os.path.join(sub_val_dirs[4], 'positive'), i + '.jpg')
            else:
                file_path = os.path.join(os.path.join(sub_val_dirs[4], 'negative'), i + '.jpg')
    #Resize img to 256 by 256
    p_img = pil.fromarray(img)
    p_img = p_img.resize((256, 256))
    img = np.asarray(p_img)
    cv2.imwrite(file_path, img)
    os.remove(file)

In [23]:
for g in range(int(0.9*num_images), 645496):
    i = ids[g]
    f = os.path.join('stage_1_train_images', i + '.dcm')
    file = rsna.extract(f)
    img_data = pydicom.read_file(file)
    window_center, window_width, intercept, slope = get_windowing(img_data)
    image = pydicom.read_file(file).pixel_array
    file_path = ''
    
    for j in sub_types:
        if j == sub_types[0]:
            img = window_image(image, 600, 2800, intercept, slope) #bone window
            
            if data.iloc[g, 1] == 1:
                file_path = os.path.join(os.path.join(sub_val_dirs[0], 'positive'), i + '.jpg')
                
            else:
                file_path = os.path.join(os.path.join(sub_val_dirs[0], 'negative'), i + '.jpg')
            
                p_img = pil.fromarray(img)
                
    p_img = p_img.resize((256, 256))
    img = np.asarray(p_img)
    cv2.imwrite(file_path, img)
    os.remove(file)


In [25]:
for g in range(645500, num_images):
    i = ids[g]
    f = os.path.join('stage_1_train_images', i + '.dcm')
    file = rsna.extract(f)
    img_data = pydicom.read_file(file)
    window_center, window_width, intercept, slope = get_windowing(img_data)
    image = pydicom.read_file(file).pixel_array
    file_path = ''
    
    for j in sub_types:
        if j == sub_types[0]:
            img = window_image(image, 600, 2800, intercept, slope) #bone window
            
            if data.iloc[g, 1] == 1:
                file_path = os.path.join(os.path.join(sub_val_dirs[0], 'positive'), i + '.jpg')
                
            else:
                file_path = os.path.join(os.path.join(sub_val_dirs[0], 'negative'), i + '.jpg')
            
            cv2.imwrite(file_path, img)
            print(str(g) + file_path)

645500/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_52aa97916.jpg
645501/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_54ae20b2e.jpg
645502/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/positive/ID_1acff1a0f.jpg
645503/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/positive/ID_c33085973.jpg
645504/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_02ec46827.jpg
645505/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_32df584f2.jpg
645506/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_d68d78849.jpg
645507/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_1f15515ad.jpg
645508/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_927d7f8c9.jpg
645509/home/wkeeley/Desktop/Brain-Hem

645583/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_8f5aad3c2.jpg
645584/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_0e62ea5fd.jpg
645585/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_8d8a2c89a.jpg
645586/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_7e0588c6e.jpg
645587/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_b71fae3ad.jpg
645588/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_0c943a304.jpg
645589/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_4f3473f86.jpg
645590/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_f83187ef1.jpg
645591/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_cca22a0e6.jpg
645592/home/wkeeley/Desktop/Brain-Hem

645667/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_c600fdf1f.jpg
645668/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_4ca794e8b.jpg
645669/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_1eb0b10ee.jpg
645670/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_56ddfac3f.jpg
645671/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_42ca3f699.jpg
645672/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_5a73297f2.jpg
645673/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_22aa30074.jpg
645674/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_4521e8d25.jpg
645675/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_0904ad378.jpg
645676/home/wkeeley/Desktop/Brain-Hem

645748/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_771bda6c9.jpg
645749/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_c67daa364.jpg
645750/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_0ef66d608.jpg
645751/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_88d4edc8a.jpg
645752/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_863806ef8.jpg
645753/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_7992d1669.jpg
645754/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_0f52abcb1.jpg
645755/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_c8afcbb6c.jpg
645756/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_565ae6c8b.jpg
645757/home/wkeeley/Desktop/Brain-Hem

645832/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_417e77b1a.jpg
645833/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_c72ee4b3d.jpg
645834/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_bdb5977d7.jpg
645835/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_b13084a64.jpg
645836/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_3cc35ca39.jpg
645837/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_f2c64a0df.jpg
645838/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_1a353a009.jpg
645839/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_2d8fb0992.jpg
645840/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_54e9d628a.jpg
645841/home/wkeeley/Desktop/Brain-Hem

...

673872/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_cf342e523.jpg
673873/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_4df735b00.jpg
673874/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_68f9d1472.jpg
673875/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_b3ffce0b6.jpg
673876/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_e06f45c0f.jpg
673877/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_bbd393979.jpg
673878/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_359d6359e.jpg
673879/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_b8c9d58dc.jpg
673880/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_7bb54ceee.jpg
673881/home/wkeeley/Desktop/Brain-Hem

673953/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/positive/ID_8db5e5f6e.jpg
673954/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_483b7e349.jpg
673955/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/positive/ID_fd1596030.jpg
673956/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_0fe874eed.jpg
673957/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_0a12e696a.jpg
673958/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_f4e35a28e.jpg
673959/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_c7601d297.jpg
673960/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_b1880731e.jpg
673961/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_c0bdf13d6.jpg
673962/home/wkeeley/Desktop/Brain-Hem

674031/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_6e82f2c13.jpg
674032/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_79f10b37c.jpg
674033/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_9e13cbecd.jpg
674034/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_a7e60ee18.jpg
674035/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_049fc1444.jpg
674036/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_8f4b0ef25.jpg
674037/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_da5eb51c7.jpg
674038/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_228055a78.jpg
674039/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_635f084fc.jpg
674040/home/wkeeley/Desktop/Brain-Hem

674114/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_f25b72e26.jpg
674115/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_921551ac7.jpg
674116/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_63b0ffff1.jpg
674117/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_149d1aa26.jpg
674118/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_63b6da6be.jpg
674119/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_7782766a8.jpg
674120/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_18da5a710.jpg
674121/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_432ffd42e.jpg
674122/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_4e24632d6.jpg
674123/home/wkeeley/Desktop/Brain-Hem

674195/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_fb2355305.jpg
674196/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_2c7fab6df.jpg
674197/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_c2f2b8ea0.jpg
674198/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_e7ac48e64.jpg
674199/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_0a1e965bd.jpg
674200/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_854773637.jpg
674201/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_68f5b0804.jpg
674202/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_d2582ff2f.jpg
674203/home/wkeeley/Desktop/Brain-Hemorrhaging/Images/Validation_Images/epidural/negative/ID_083909e75.jpg
674204/home/wkeeley/Desktop/Brain-Hem